In [45]:
import logging
import threading
import time

In [46]:
def thread_function(name, secondary_id):
    logging.info(f'Thread %s:%s is starting' % (name, secondary_id))
    time.sleep(2)
    logging.info(f'Thread %s:%s is ending' % (name, secondary_id))

In [47]:
format = "%(asctime)s: %(message)s"
logging.basicConfig(format=format, level=logging.INFO, datefmt="%H:%M:%S")

In [58]:
def parent_method():
    logging.info("Main    : before creating thread")
#     x = threading.Thread(target=thread_function, args=('one', 1), daemon=True)
    x = threading.Thread(target=thread_function, args=('one', 1))
    logging.info("Main    : before running thread")
    x.start()
    logging.info("Main    : wait for the thread to finish")
#     x.join()
    logging.info("Main    : all done")
    
# Note that daemon setting won't work in ipython. If I had used
# __name__ == '__main__' and then invoked the script, the use of daemon setting can be demo'ed

In [60]:
parent_method()

18:35:14: Main    : before creating thread
18:35:14: Main    : before running thread
18:35:14: Thread one:1 is starting
18:35:14: Main    : wait for the thread to finish
18:35:16: Thread one:1 is ending
18:35:16: Main    : all done


In [123]:
import logging
import threading
import time

def thread_function(name):
    logging.info("Thread %s: starting", name)
    time.sleep(2)
    logging.info("Thread %s: finishing", name)

format = "%(asctime)s: %(message)s"
logging.basicConfig(format=format, level=logging.INFO,
                    datefmt="%H:%M:%S")

threads = list()
for index in range(3):
    logging.info("Main    : create and start thread %d.", index)
    x = threading.Thread(target=thread_function, args=(index,))
    threads.append(x)
    x.start()

for index, thread in enumerate(threads):
    logging.info("Main    : before joining thread %d.", index)
    thread.join()
    logging.info("Main    : thread %d done", index)

10:00:32: Main    : create and start thread 0.
10:00:32: Thread 0: starting
10:00:32: Main    : create and start thread 1.
10:00:32: Thread 1: starting
10:00:32: Main    : create and start thread 2.
10:00:32: Thread 2: starting
10:00:32: Main    : before joining thread 0.
10:00:34: Thread 0: finishing
10:00:34: Main    : thread 0 done
10:00:34: Main    : before joining thread 1.
10:00:34: Thread 1: finishing
10:00:34: Main    : thread 1 done
10:00:34: Main    : before joining thread 2.
10:00:34: Thread 2: finishing
10:00:34: Main    : thread 2 done


In [124]:
# Using a thread pool executor
import concurrent.futures

# [rest of code]

format = "%(asctime)s: %(message)s"
logging.basicConfig(format=format, level=logging.INFO,
                    datefmt="%H:%M:%S")

with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
    executor.map(thread_function, range(3))

10:00:38: Thread 0: starting
10:00:38: Thread 1: starting
10:00:38: Thread 2: starting
10:00:40: Thread 0: finishing
10:00:40: Thread 2: finishing
10:00:40: Thread 1: finishing


In [151]:
# class FakeDatabase:
#     def __init__(self):
#         self.value = 0

#     def update(self, name):
#         logging.info("Thread %s: starting update", name)
#         local_copy = self.value
#         local_copy += 1
#         time.sleep(0.1)
#         self.value = local_copy
#         logging.info("Thread %s: finishing update", name)

In [153]:
class FakeDatabase:
    def __init__(self):
        self.value = 0
        self._lock = threading.Lock()

    def update(self, name):
        with self._lock:
            logging.info("Thread %s: starting update", name)
            local_copy = self.value
            local_copy += 1
            time.sleep(0.1)
            self.value = local_copy
            logging.info("Thread %s: finishing update", name)

In [154]:
format = "%(asctime)s: %(message)s"
logging.basicConfig(format=format, level=logging.INFO,
                    datefmt="%H:%M:%S")

database = FakeDatabase()
logging.info("Testing update. Starting value is %d.", database.value)
with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
    for index in range(2):
        executor.submit(database.update, index)
logging.info("Testing update. Ending value is %d.", database.value)

10:44:44: Testing update. Starting value is 0.
10:44:44: Thread 0: starting update
10:44:44: Thread 0: finishing update
10:44:44: Thread 1: starting update
10:44:45: Thread 1: finishing update
10:44:45: Testing update. Ending value is 2.


In [156]:
import threading

l = threading.RLock()
print("before first acquire")
l.acquire()
print("before second acquire")
l.acquire()
print("acquired lock twice")
l.release()
l.release()

before first acquire
before second acquire
acquired lock twice


In [183]:
import random

SENTINEL = 'SENTINEL'

def producer(pipeline):
    for iteration in range(10):
        message = random.randint(1,100)
        print("Producer generating message {}".format(message))
        pipeline.set_message(message, "Producer")
        print("Producer sent message {}".format(message))
        
    pipeline.set_message(SENTINEL, "Producer")

In [184]:
def consumer(pipeline):
    message = None
    while message is not SENTINEL:
        message = pipeline.get_message("Consumer")        
        print("Consumer received message {}".format(message))

In [185]:
class Pipeline:
    def __init__(self):
        self.message = None
        self.producer_lock = threading.Lock()
        self.consumer_lock = threading.Lock()
        self.consumer_lock.acquire()
        
    def set_message(self, message, agent):
        self.producer_lock.acquire()
        self.message = message
        self.consumer_lock.release()
        
    def get_message(self, agent):
        self.consumer_lock.acquire()
        message = self.message
        self.producer_lock.release()
        return message

In [186]:
if __name__ == "__main__":
    pipeline = Pipeline()
    with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
        executor.submit(producer, pipeline)
        executor.submit(consumer, pipeline)

Producer generating message 95
Producer sent message 95
Producer generating message 64
Consumer received message 95
Producer sent message 64Consumer received message 64
Producer generating message 64
Producer sent message 64
Producer generating message 73

Consumer received message 64
Producer sent message 73
Producer generating message 29
Consumer received message 73
Producer sent message 29Consumer received message 29

Producer generating message 2
Producer sent message 2
Consumer received message 2Producer generating message 84
Producer sent message 84
Producer generating message 31

Consumer received message 84
Producer sent message 31Consumer received message 31

Producer generating message 7
Producer sent message 7
Producer generating message 67
Consumer received message 7Producer sent message 67

Consumer received message 67
Consumer received message SENTINEL


In [205]:
import concurrent.futures
import logging
import queue
import random
import threading
import time

def producer(queue, event):
    """Pretend we're getting a number from the network."""
    while not event.is_set():
        message = random.randint(1, 101)
        logging.info("Producer got message: %s", message)
        queue.put(message)

    logging.info("Producer received event. Exiting")

def consumer(queue, event):
    """Pretend we're saving a number in the database."""
    while not event.is_set() or not queue.empty():
        message = queue.get()
        logging.info(
            "Consumer storing message: %s (size=%d)", message, queue.qsize()
        )

    logging.info("Consumer received event. Exiting")

if __name__ == "__main__":
    format = "%(asctime)s: %(message)s"
    logging.basicConfig(format=format, level=logging.INFO,
                        datefmt="%H:%M:%S")

    pipeline = queue.Queue(maxsize=10)
    event = threading.Event()
    with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
        executor.submit(producer, pipeline, event)
        executor.submit(consumer, pipeline, event)

        time.sleep(0.1)
        logging.info("Main: about to set event")
        event.set()

12:32:51: Producer got message: 46
12:32:51: Producer got message: 41
12:32:51: Producer got message: 27


Sending message 74
Receiving message 11
Receiving message 46


Sending message 47

Receiving message 54
Receiving message 56
Receiving message 57
Receiving message 66
Receiving message 7
Sending message 100
Sending message 14
Sending message 21
Sending message 81
Sending message 39
Receiving message 76
Receiving message 16
Receiving message 30
Receiving message 66Receiving message 18
Receiving message 100

Receiving message 5
Sending message 47Sending message 46Sending message 6
Sending message 36
Sending message 35
Sending message 41

Receiving message 57

Sending message 17
Sending message 48
Sending message 62
Sending message 25
Receiving message 36
Receiving message 10
Receiving message 14
Receiving message 21
Receiving message 81
Receiving message 39
Sending message 66
Receiving message 86
Receiving message 27
Receiving message 88
Receiving message 70
Receiving message 67
Receiving message 47
Receiving message 6
Receiving message 36
Receiving message 35
Receiving message 41
Sendin

12:32:51: Consumer storing message: 46 (size=1)
12:32:51: Consumer storing message: 41 (size=1)
12:32:51: Producer got message: 7
12:32:51: Consumer storing message: 27 (size=0)


Sending message 98
Sending message 64
Receiving message 59Receiving message 45
Receiving message 49
Receiving message 63
Receiving message 66
Receiving message 3
Sending message 95
Sending message 86
Sending message 60Sending message 6
Receiving message 75
Receiving message 99Sending message 21

Receiving message 39

Sending message 80
Sending message 27
Receiving message 9
Receiving message 42
Receiving message 60
Receiving message 77
Receiving message 90
Receiving message 66
Receiving message 1
Receiving message 94
Sending message 69
Sending message 34Receiving message 98
Receiving message 95
Receiving message 86
Receiving message 60
Receiving message 80


Sending message 59

Receiving message 84
Sending message 98
Sending message 17
Sending message 18
Sending message 10
Sending message 34

Receiving message 98Receiving message 74Receiving message 15
Receiving message 64

Receiving message 12
Receiving message 3
Receiving message 6
Receiving message 69
Sending message 55
Sending mess

12:32:51: Consumer storing message: 7 (size=0)
12:32:51: Producer got message: 87
12:32:52: Producer got message: 84
12:32:52: Main: about to set event


...

Sending message 94Receiving message 96
Sending message 97
Receiving message 25Receiving message 40
Receiving message 71
Sending message 2

Receiving message 58
Receiving message 95
Receiving message 9
Receiving message 88
Receiving message 90
Receiving message 99
Receiving message 6
Receiving message 51
Receiving message 38

Sending message 41
Sending message 41
Sending message 39
Sending message 69
Sending message 42

Receiving message 69
Receiving message 71
Receiving message 53
Receiving message 58
Receiving message 100
Receiving message 11Sending message 84Sending message 74
Receiving message 16
Receiving message 91
Receiving message 94
Receiving message 97
Receiving message 2

Sending message 43
Sending message 90
Sending message 56
Sending message 72
Sending message 59
Sending message 39
Sending message 1
Sending message 22
Sending message 21
Sending message 58
Sending message 4
Receiving message 74
Receiving message 55Receiving message 12Receiving message 1Sending message 43
Sen

Sending message 69
Receiving message 3
Receiving message 58
Receiving message 91
Receiving message 90Receiving message 75

Receiving message 43
Receiving message 59
Receiving message 83
Receiving message 66
Receiving message 62
Receiving message 2
Receiving message 48
Receiving message 63
Receiving message 9
Receiving message 50

Sending message 68
Sending message 13
Sending message 56
Sending message 3

Sending message 28Receiving message 74
Sending message 14
Sending message 99
Sending message 80
Sending message 22
Sending message 39
Sending message 29
Sending message 28
Sending message 78
Sending message 40
Sending message 50
Sending message 49Receiving message 78
Receiving message 69
Receiving message 68
Receiving message 13
Receiving message 56

Sending message 51
Sending message 89
Sending message 26
Sending message 61
Receiving message 43Sending message 69Sending message 41
Receiving message 49
Receiving message 51
Receiving message 89
Receiving message 26

Receiving message 3



Sending message 34Sending message 100
Sending message 99

Receiving message 80
Receiving message 95

Sending message 90
Sending message 8
Sending message 95
Sending message 18
Sending message 58
Sending message 19

Sending message 7
Sending message 29
Sending message 62
Sending message 66
Sending message 87
Sending message 89
Sending message 36
Sending message 90
Sending message 88
Receiving message 41Receiving message 91Receiving message 1
Sending message 97
Receiving message 99
Receiving message 90
Receiving message 8
Receiving message 95
Receiving message 18

Sending message 18
Sending message 93
Sending message 17
Sending message 98
Sending message 8
Sending message 22Receiving message 97
Receiving message 18
Receiving message 93
Receiving message 17
Receiving message 98


Receiving message 100
Receiving message 7
Receiving message 29
Receiving message 62
Receiving message 66
Receiving message 87
Receiving message 89Sending message 70
Sending message 20
Receiving message 60
Receivi

Receiving message 75Sending message 68Sending message 84
Sending message 95
Sending message 30
Sending message 62
Sending message 67
Receiving message 81
Receiving message 65
Receiving message 63
Receiving message 84
Receiving message 84
Receiving message 95
Receiving message 30
Receiving message 62
Sending message 39Receiving message 56Sending message 91Receiving message 67


Receiving message 100

Receiving message 69
Receiving message 78
Receiving message 39

Sending message 46
Sending message 95
Sending message 44
Sending message 40
Receiving message 89Sending message 88
Sending message 99
Sending message 87
Sending message 7
Sending message 2

Sending message 29
Receiving message 68Sending message 44
Sending message 99
Sending message 51
Sending message 80
Sending message 30
Receiving message 39
Sending message 53
Receiving message 88
Receiving message 99
Receiving message 87
Receiving message 7
Receiving message 2

Receiving message 44
Receiving message 99
Receiving message 51
Re


Sending message 91
Sending message 71
Sending message 96
Sending message 87
Sending message 69
Receiving message 60Receiving message 80Sending message 3
Receiving message 56
Receiving message 14
Receiving message 40
Receiving message 34

Receiving message 91
Receiving message 71
Receiving message 96
Receiving message 87

Sending message 93
Sending message 39
Sending message 25
Sending message 64
Sending message 67Receiving message 3
Receiving message 93
Receiving message 39
Receiving message 25
Receiving message 64
Sending message 84
Sending message 9
Sending message 7
Sending message 32
Sending message 35
Sending message 95

Sending message 100
Sending message 79
Sending message 66
Sending message 21Receiving message 69Receiving message 67
Sending message 60
Sending message 74
Receiving message 9
Receiving message 7
Receiving message 32
Sending message 78
Sending message 32
Sending message 36


Receiving message 35Receiving message 84Sending message 32
Receiving message 95
Receiving 